Original implementation of Contrastive-sc method
(https://github.com/ciortanmadalina/contrastive-sc)

In [4]:
import sys
sys.path.append("..")
import argparse
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score
from sklearn.cluster import KMeans
from sklearn import metrics

import torch
import torch.nn as nn
import copy
from tqdm.notebook import tqdm
import models
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import st_loss
import time
import h5py
import scipy as sp
import scanpy.api as sc
from collections import Counter
import random
import utils
import loop
import pickle

import train
import os
import glob2
plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
path = "../"
category = "real_data"
files = glob2.glob(f'{path}{category}/*.h5')
files = [f[len(f"'{path}{category}"):-3] for f in files]
files

['Quake_Smart-seq2_Trachea',
 'Quake_Smart-seq2_Diaphragm',
 'Quake_10x_Spleen',
 'Young',
 'mouse_ES_cell',
 'Adam',
 'Quake_10x_Bladder',
 'Quake_Smart-seq2_Lung',
 'Quake_10x_Limb_Muscle',
 'worm_neuron_cell',
 'mouse_bladder_cell',
 'Romanov',
 'Quake_Smart-seq2_Limb_Muscle',
 'Muraro',
 '10X_PBMC']

In [21]:
dataset = 'data_-1c4'
category = "balanced_data"

In [26]:
X = np.arange(10).reshape(2, 5)

In [31]:
df = pd.DataFrame()
dropout = 0.9
lr = 1e-5
layers = [200, 40, 60]
temperature = 0.07
nb_genes = 500
data_mat = h5py.File(f"{path}R/simulated_data/{category}/{dataset}.h5", "r")
X_o = np.array(data_mat['X'])
Y_o = np.array(data_mat['Y'])
cluster_number = np.unique(Y_o).shape[0]

X_o = train.preprocess(X_o, nb_genes=nb_genes)
for factor in [1, 5, 10, 50, 100, 150]:
    X = np.tile(X_o.T, factor).T
    Y= np.tile(Y_o.T, factor).T
    print(">> ",X_o.shape, X.shape)
    for run in range(3):
        torch.manual_seed(run)
        torch.cuda.manual_seed_all(run)
        np.random.seed(run)
        random.seed(run)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        start = time.time()
        dresults = train.run(X,
                             cluster_number,
                             dataset,
                             Y=Y,
                             nb_epochs=30,
                             lr=lr,
                             temperature=temperature,
                             dropout=dropout,
                             evaluate=False,
                             n_ensemble=1,
                             layers=layers,
                             save_to=f"{path}output/{category}/{run}/",
                             save_pred = True)
        elapsed = time.time()-start
        dresults["nb_cells"] = X.shape[0]
        dresults["time"] = elapsed
    print(f"{elapsed}", end = " ")
#         print(f"# {temperature}, {dropout}, {lr}, {layers}", 
#               dresults.get('COMBINED_kmeans_ari', ""),
#               dresults.get('COMBINED_leiden_ari', ""), dresults.get('kmeans_ari_0',""),
#               dresults.get('leiden_ari_0', ""))
    df = df.append(dresults, ignore_index=True)

    df.to_pickle(f"{path}output/pickle_results/{category}/{category}_scalability.pkl")

(1000, 2500) (1000, 2500) keeping 500 genes
>>  (1000, 500) (1000, 500)
0.7961325645446777 >>  (1000, 500) (5000, 500)
4.042118310928345 >>  (1000, 500) (10000, 500)
8.111743688583374 >>  (1000, 500) (50000, 500)
41.13520407676697 >>  (1000, 500) (100000, 500)
84.65198707580566 >>  (1000, 500) (150000, 500)
124.66856002807617 

In [ ]:
df.mean()